# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd

import sqlite3
from sqlalchemy import create_engine

import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import TransformerMixin
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import joblib

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mfuza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
conn = sqlite3.connect('./data/Disaster.db')

In [3]:
# load data from database
engine = create_engine('sqlite:///data/Disaster.db')
df = pd.read_sql('select * from disaster',conn)
df.head()
X = df['message']
Y = df.loc[:,'related':]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_token = []
    for token in tokens:
        clean_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_token.append(clean_tok)
    
    return clean_token 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
  ('vect', CountVectorizer(tokenizer=tokenize)),
  ('tfid',TfidfTransformer()),
  ('rf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X.values,Y.values)
pipeline.fit(X_train,Y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001E5AE0E83A0>)),
                ('tfid', TfidfTransformer()),
                ('rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [7]:
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
for i,col in enumerate(Y.columns):
  print(col)
  print(classification_report(Y_test[i],Y_pred[i]))

related
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        31
           1       1.00      0.40      0.57         5

    accuracy                           0.92        36
   macro avg       0.96      0.70      0.76        36
weighted avg       0.92      0.92      0.90        36

request
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        33
           1       1.00      0.33      0.50         3

    accuracy                           0.94        36
   macro avg       0.97      0.67      0.74        36
weighted avg       0.95      0.94      0.93        36

offer
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.60      0.75         5

    accuracy                           0.94        36
   macro avg       0.97      0.80      0.86        36
weighted avg       0.95      0.94      0.94        3

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# parameters = {
#   'rf__estimator__n_estimators': [10, 50,100],
#   'rf__estimator__min_samples_split': [2, 3, 4],
# }

# cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
# cv.fit(X_train, Y_train)

In [11]:
# cv.best_params_

Best params:
- min_sample_split : 2,
- n_estimators: 100

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
class DenseTransformer(TransformerMixin):
  
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [13]:
pipeline_nb = Pipeline([
  ('vect', CountVectorizer(tokenizer=tokenize)),
  ('tfid',TfidfTransformer()),
  ('to_dense', DenseTransformer()),
  ('rf', MultiOutputClassifier(GaussianNB()))
])

In [14]:
pipeline_nb.fit(X_train,Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001E5AE0E83A0>)),
                ('tfid', TfidfTransformer()),
                ('to_dense',
                 <__main__.DenseTransformer object at 0x000001E5B5146220>),
                ('rf', MultiOutputClassifier(estimator=GaussianNB()))])

In [15]:
Y_pred = pipeline_nb.predict(X_test)

In [16]:
for i,col in enumerate(Y.columns):
  print(classification_report(Y_test[i],Y_pred[i]))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        31
           1       1.00      0.20      0.33         5

    accuracy                           0.89        36
   macro avg       0.94      0.60      0.64        36
weighted avg       0.90      0.89      0.86        36

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00         3

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        31
           1       1.00      0.20      0.33         5

    accuracy                           0.89        36
   macro avg       0.94      0.60      0.64        36
weighted avg       0.90      0.89      0.86        36

              preci

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [19]:
joblib.dump(pipeline,"./models/model_rf.pkl")

['./models/model_rf.pkl']

In [20]:
joblib.dump(pipeline_nb,'./models/model_nb.pkl')

['./models/model_nb.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.